In [1]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

base = np.array([[20],[30],[50],[120],[22],[63],[102],[34],[87],[12],[25],[128],[23],[77],[56]])
target = np.array([[50],[70],[110],[250],[54],[136],[214],[78],[184],[34],[60],[266],[56],[166],[122]])
scaler = MinMaxScaler()
base_scaled = scaler.fit_transform(base)
base_unscaled = scaler.inverse_transform(base_scaled)

target_scaled = scaler.fit_transform(target[:5])
target_unscaled = scaler.inverse_transform(target_scaled)

num_clusters = 3
#print(target_scaled)

c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [2]:

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
kmeans.fit(base_scaled)

base_centroids = kmeans.cluster_centers_
#labels = kmeans.labels_
    
print(kmeans.cluster_centers_)
print(kmeans.inertia_)

[[0.47068966]
 [0.10098522]
 [0.90229885]]
0.1175159956967329


In [3]:
def cal_centroids(alpha,beta):

    target_centroids = []
    for i in range(len(base_centroids)):
        val = alpha*base_centroids[i][0] + beta
        target_centroids.append([val])

    target_centroids = np.array(target_centroids)
    print(target_centroids)
    return target_centroids
    #print(target_centroids)

In [10]:
def cal_gradient():
    gradient_alpha = 0
    gradient_beta = 0
    for i in range(0,3):
        gradient = 0
        print(reframed_centroids[i][0])
        print(centroids[i])
        for member in clusters[i]:
            print(member[0])
            gradient = reframed_centroids[i][0] - member[0]

        gradient_alpha = gradient_alpha + gradient*reframed_centroids[i][0]
        gradient_beta = gradient_beta + gradient
        print(clusters[i])
        print(gradient_alpha)
        print(gradient_beta)
        print("******")

In [5]:
sum_m = 0
sum_d = 0
for i in range(len(base_centroids)):
    sum_m = sum_m + base_centroids[i][0]
    
avg_m = sum_m/len(base_centroids)

for i in range(len(target_scaled)):
    sum_d = sum_d + target_scaled[i][0]

print(sum_m,sum_d)
avg_d = sum_d/len(target_scaled)

alpha = avg_m/avg_d
beta = 0
print(alpha)
        


1.4739737274220033 1.4200000000000002
1.7300161119976563


In [6]:
reframed_centroids =  cal_centroids(alpha,beta)
kmeans = KMeans(n_clusters=num_clusters, init=reframed_centroids, max_iter=1)
kmeans.fit(target_scaled)
old_error = kmeans.inertia_
print(old_error)

# while(1):
    
#     alpha = alpha - 0.01*
#     beta = beta + 1
#     kmeans = KMeans(n_clusters=num_clusters, init=cal_parameters(alpha,beta), max_iter=1)
#     kmeans.fit(target_unscaled)
#     new_error = kmeans.inertia_
#     print(old_error,new_error)
#     if(new_error < old_error):
#         old_error = new_error
                
#     else:
#         break;
    
    

#     #base_centroids = kmeans.cluster_centers_
#     #labels = kmeans.labels_

#     #print(kmeans.cluster_centers_)
#     print(old_error,new_error)

[[0.81430069]
 [0.17470606]
 [1.56099155]]
0.05630000000000002


c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


In [7]:
from collections import Counter

centroids = kmeans.cluster_centers_
labels = kmeans.labels_

c = Counter(labels)

clusters = []
for i in range(0,3):
    clusters.append([])
    
#print(kmeans.cluster_centers_)
for i in range(len(target_scaled)):
    #print("coordinate:",X[i], "label:", labels[i], "keyword:", keywords[i])
    clusters[labels[i]].append(target_scaled[i])

#unscaled_centroids = scaler.inverse_transform(centroids)
gradient_alpha = 0
gradient_beta = 0
for i in range(0,3):
    gradient = 0
    print(reframed_centroids[i][0])
    print(centroids[i])
    for member in clusters[i]:
        print(member[0])
        gradient = reframed_centroids[i][0] - member[0]
    
    gradient_alpha = gradient_alpha + gradient*reframed_centroids[i][0]
    gradient_beta = gradient_beta + gradient
    print(clusters[i])
    print(gradient_alpha)
    print(gradient_beta)
    print("******")

0.8143006871988968
[1.]
1.0
[array([1.])]
-0.15121507802630121
-0.1856993128011032
******
0.17470606057119187
[0.105]
0.0
0.10000000000000003
0.30000000000000004
0.020000000000000018
[array([0.]), array([0.1]), array([0.3]), array([0.02])]
-0.12418699163742009
-0.03099325222991134
******
1.5609915493312188
[1.]
[]
-0.12418699163742009
-0.03099325222991134
******


In [8]:
alpha = alpha-1*gradient_alpha
beta = beta-1*gradient_beta
print(alpha,beta)

1.8542031036350763 0.03099325222991134


In [9]:
reframed_centroids =  cal_centroids(alpha,beta)
kmeans = KMeans(n_clusters=num_clusters, init=reframed_centroids, max_iter=1)
kmeans.fit(target_scaled)
old_error = kmeans.inertia_
print(old_error)

[[0.90374747]
 [0.21824036]
 [1.70403858]]
0.03845


c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
